In [4]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [5]:
data_train=pd.read_csv('data_train.txt',index_col=[0],header=0)
data_valid=pd.read_csv('data_valid.txt',index_col=[0],header=0)

In [7]:
data_train.columns

Index(['user_id', 'weibo_id', 'time', 'repost', 'comments', 'likes', 'tfidf',
       'text', 'number_in_train', 'forward_max', 'comment_max', 'like_max',
       'forward_min', 'comment_min', 'like_min', 'forward_mean',
       'comment_mean', 'like_mean', 'forward_more_ave_pr',
       'comment_more_ave_pr', 'like_more_ave_pr', 'max_f/l', 'max_c/l',
       'min_f/l', 'min_c/l', 'mean_f/l', 'mean_c/l', 'time_weekday',
       'time_weekend', 'time_hour', 'panduan', 'length_all', 'length_chinese',
       'english', 'non_ch', 'sharing', 'auto', 'interaction', 'book',
       'mention', 'vote', 'lottery', 'emoji', 'video', 'http', 'app', 'title',
       'ad', 'hotwords', 'keywords', 'stock', 'is_noise'],
      dtype='object')

In [80]:
#先筛选两个子集，检测代码
train_subset=data_train.loc[:,['repost','comments','likes','tfidf','number_in_train','forward_max','time_weekend','length_chinese','sharing','hotwords', 'keywords','is_noise']]
valid_subset=data_valid.loc[:,['repost','comments','likes','tfidf','number_in_train','forward_max','time_weekend','length_chinese','sharing','hotwords', 'keywords','is_noise']]

In [81]:
def decision_tree(data_train,data_test,depth,leaf_num,split_num):
    data_test['repost_hat']=0
    data_test['comments_hat']=0
    data_test['likes_hat']=0
    
    #噪音数据预测值均为0,因此仅处理非噪音数据
    train=data_train[data_train['is_noise']==0]
    test=data_test[data_test['is_noise']==0]
    
    #分别对转发，评论和点赞建立三棵树
    tree_repost = DecisionTreeRegressor(criterion='mse', min_samples_leaf=leaf_num,max_depth=depth,min_samples_split=split_num)
    tree_comments= DecisionTreeRegressor(criterion='mse', min_samples_leaf=leaf_num,max_depth=depth,min_samples_split=split_num)
    tree_likes = DecisionTreeRegressor(criterion='mse', min_samples_leaf=leaf_num,max_depth=depth,min_samples_split=split_num)
    
    #拟合三棵树
    regressor_train=train.drop(['repost','comments','likes','is_noise'],axis=1, inplace=False)
    repost_train=train.loc[:,['repost']]
    comments_train=train.loc[:,['comments']]
    likes_train=train.loc[:,['likes']]
    predict_repost=tree_repost.fit(regressor_train,repost_train)
    predict_comments=tree_comments.fit(regressor_train,comments_train)
    predict_likes=tree_likes.fit(regressor_train,likes_train)
    
    #预测测试集的数据
    regressor_test=test.drop(['repost','comments','likes','is_noise','repost_hat','comments_hat','likes_hat'],axis=1, inplace=False)
    repost_hat=np.round(predict_repost.predict(regressor_test),0)  #round函数只是返回四舍五入值，是浮点类型
    comments_hat=np.round(predict_comments.predict(regressor_test),0)
    likes_hat=np.round(predict_likes.predict(regressor_test),0)
    
    #将预测值赋值并设置为整数值
    data_test['repost_hat'][data_test['is_noise']==0]=repost_hat
    data_test['comments_hat'][data_test['is_noise']==0]=comments_hat
    data_test['likes_hat'][data_test['is_noise']==0]=likes_hat
    data_test['repost_hat']=data_test['repost_hat'].apply(lambda x:int(x))
    data_test['comments_hat']=data_test['comments_hat'].apply(lambda x:int(x))
    data_test['likes_hat']=data_test['likes_hat'].apply(lambda x:int(x))
    return data_test    

In [82]:
valid_subset=decision_tree(train_subset,valid_subset,10,10,10)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1]:
valid_subset.head()

NameError: name 'valid_subset' is not defined

In [84]:
def precision(data):
    data['deviation_repost']=list(map(lambda x, y: abs(x-y)/(y+5), data['repost_hat'],data['repost']))
#print (data['deviation_repost'])
    data['deviation_likes']=list(map(lambda x, y: abs(x-y)/(y+3), data['likes_hat'],data['likes']))
    #print (data['deviation_likes'])
    data['deviation_comments']=list(map(lambda x, y: abs(x-y)/(y+3), data['comments_hat'],data['comments']))
    #print (data['deviation_comments'])
    data['lcf_sum']=data['repost']+data['likes']+data['comments']
#    print (data['lcf_sum'])
    data['lcf_sum']=data['lcf_sum'].apply(lambda x: 100 if x>100 else x)
    data['precision_1_-0.8']=1-0.5*data['deviation_repost']-0.25*data['deviation_likes']-0.25*data['deviation_comments']-0.8
    #print (data['precision_1_-0.8'])
    data.loc[data['precision_1_-0.8']<=0,'sgn']=0
    data.loc[data['precision_1_-0.8']>0,'sgn']=1
#    print (data['sgn'])
    precision_=sum((data['lcf_sum']+1)*data['sgn'])/sum(data['lcf_sum']+1)

    
    return precision_

In [85]:
precision(valid_subset)

0.24510481697549288

In [76]:
valid_subset[['repost_hat','comments_hat','likes_hat']][(valid_subset['repost_hat']!=0)&(valid_subset['comments_hat']!=0)]

,repost_hat,comments_hat,likes_hat
5,5,2,0
14,5,2,0
56,5,2,0
57,5,2,0
73,5,2,0
97,5,2,0


In [39]:
(valid_subset['repost_hat']==0) & (valid_subset['comments_hat']==0)

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
71     True
72     True
73     True
74     True
75     True
76     True
77     True
78     True
79     True
80     True
81     True
82     True
83     True
84     True
85     True
86     True
87     True
88     True
89     True
90     True
91     True
92     True
93     True
94     True
95     True
96     True
97     True
98     True
99     True
100    True
Length: 101, dtype: bool